In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [4]:
seed_value = 0

np.random.seed(seed_value)
tf.set_random_seed(seed_value)

In [5]:
class CustomDense(keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(name='kernel',
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(name='kernel',
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)